In [3]:
%pip install 'vanna[chromadb,openai,postgres]'

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 964.9/964.9 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 10.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.0 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 5.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 8.5 MB/s eta 0:00:00ta 0:00:01
  Created wheel for chroma-hnswlib: filename=chroma_hnswlib-0.7.6-cp313-cp313-linux_x86_64.whl size=227072 sha256=097b27f7e848f8eee173fe4f4608d29b6d56c678e3cb8867bf96e7269a7bbc7c
  Stored in directory: /home/user/.cache/pip

In [1]:
from src.core.settings import DB_USER, DB_HOST, DB_PASS, DB_PORT, DB_NAME, VANNA_MODEL_NAME, VANNA_API_KEY
from vanna.remote import VannaDefault
vn = VannaDefault(model=VANNA_MODEL_NAME, api_key=VANNA_API_KEY)
vn.connect_to_postgres(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASS, port=DB_PORT)

In [23]:
df_information_schema = vn.run_sql("SELECT * FROM INFORMATION_SCHEMA.COLUMNS")
my_tables = ['articles', 'compounds', 'article_compound', 'assays']
df_schema = df_information_schema[df_information_schema['table_name'].isin(my_tables)]
plan = vn.get_training_plan_generic(df_schema)
plan.get_summary()

['Train on Information Schema: pubchem.public articles',
 'Train on Information Schema: pubchem.public assays',
 'Train on Information Schema: pubchem.public article_compound',
 'Train on Information Schema: pubchem.public compounds']

In [27]:
vn.train(plan=plan)

In [9]:
vn.train(documentation="To find the most frequently mentioned compound, join compounds and article_compound tables.")
vn.train(documentation="Example: SELECT c.name, COUNT(ac.article_id) FROM compounds c JOIN article_compound ac ON c.id = ac.compound_id GROUP BY c.name ORDER BY COUNT(ac.article_id) DESC;")


Adding documentation....
Adding documentation....


''

In [ ]:
from vanna.flask import VannaFlaskApp
app = VannaFlaskApp(vn)
app.run()

Your app is running at:
http://localhost:8084
 * Serving Flask app 'vanna.flask'
 * Debug mode: on


In [31]:
# 1. Find all compounds with molecular weight < 500
vn.train(
    question="Find all compounds with molecular weight less than 500",
    sql="""
    SELECT id, name, molecular_weight
    FROM compounds
    WHERE molecular_weight < 500
    """
)

# 2. Find all compounds that were active in a given BioAssay
vn.train(
    question="Find all compounds that were active in BioAssay X",
    sql="""
    SELECT c.id, c.name, a.assay_id, a.activity_outcome
    FROM compounds c
    JOIN assays a ON c.id = a.compound_id
    WHERE a.assay_id = 'X' AND a.activity_outcome = 'active'
    """
)

# 3. List all articles that mention a specific compound
vn.train(
    question="Which articles mention compound 'Imatinib'?",
    sql="""
    SELECT ar.id, ar.title, ar.journal, ar.disease_area
    FROM articles ar
    JOIN article_compound ac ON ar.id = ac.article_id
    JOIN compounds c ON ac.compound_id = c.id
    WHERE c.name ILIKE 'Imatinib'
    """
)

# 4. Retrieve all compounds studied in oncology-related articles
vn.train(
    question="List compounds studied in oncology articles",
    sql="""
    SELECT DISTINCT c.id, c.name, ar.title
    FROM compounds c
    JOIN article_compound ac ON c.id = ac.compound_id
    JOIN articles ar ON ac.article_id = ar.id
    WHERE ar.disease_area ILIKE 'oncology'
    """
)

# 5. Find compounds that pass Lipinski’s rule of five
vn.train(
    question="Which compounds pass Lipinski's rule of five?",
    sql="""
    SELECT id, name, molecular_weight, logp, tpsa
    FROM compounds
    WHERE lipinski_pass = TRUE
    """
)

# 6. Get assays with potency values below 10 uM
vn.train(
    question="Which assays report potency values below 10 uM?",
    sql="""
    SELECT assay_id, compound_id, potency_value, potency_unit
    FROM assays
    WHERE potency_value < 10 AND potency_unit = 'uM'
    """
)

# 7. Show all compounds and their activity outcomes against a given target
vn.train(
    question="Show compounds tested against EGFR and their outcomes",
    sql="""
    SELECT c.name, a.assay_id, a.activity_outcome, a.potency_value, a.potency_unit
    FROM compounds c
    JOIN assays a ON c.id = a.compound_id
    WHERE a.target_name ILIKE 'EGFR'
    """
)


'845dbe085f0608a3122333cab4202b8d-sql'

In [32]:
vn.add_documentation(
    documentation="Articles are scientific publications indexed by PubMed. Each article may reference multiple compounds."
)

vn.add_documentation(
    documentation="Compounds represent small molecules or drugs, enriched with PubChem properties such as molecular weight, logP, TPSA, and Lipinski's rule compliance."
)

vn.add_documentation(
    documentation="The article_compound table is a many-to-many relationship linking articles and compounds, with optional context text describing how the compound is mentioned."
)

vn.add_documentation(
    documentation="Assays represent experimental tests of compounds. Each assay has an outcome (active, inactive, inconclusive, etc.), a target name, and potency values with units."
)

vn.add_documentation(
    documentation="Lipinski's rule of five is a heuristic to evaluate drug-likeness. If lipinski_pass is TRUE, the compound is considered likely to be orally active."
)

vn.add_documentation(
    documentation="The disease_area field in articles indicates the therapeutic area of research, such as oncology, cardiology, or neurology."
)

'3458348-doc'